<a href="https://colab.research.google.com/github/ItsJimYang/Cifar10/blob/main/Cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# dataset has PILImage images of range [0, 1]. 
# We transform them to Tensors of normalized range [-1, 1]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train = True, transform = transform, download = True)

test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False, transform = transform, download = True)

train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 4, shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = 4, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

num_epochs = 4
batch_size = 4
learning_rate = 0.001


class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view (-1, 16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

#step up
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

#training
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate (train_loader):
    images = images.to(device)
    labels = labels.to(device)

    output = model(images)
    loss = criterion(output, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')
# PATH = './cnn.pth'
# torch.save(model.state_dict(), PATH)

#testing
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # max returns (value ,index)
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy of the network: {acc} %')

    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print(f'Accuracy of {classes[i]}: {acc} %')




Files already downloaded and verified
Files already downloaded and verified
Epoch [1/4], Step [2000/12500], Loss: 2.2917
Epoch [1/4], Step [4000/12500], Loss: 2.3383
Epoch [1/4], Step [6000/12500], Loss: 2.2946
Epoch [1/4], Step [8000/12500], Loss: 2.3198
Epoch [1/4], Step [10000/12500], Loss: 2.2997
Epoch [1/4], Step [12000/12500], Loss: 2.2726
Epoch [2/4], Step [2000/12500], Loss: 2.3104
Epoch [2/4], Step [4000/12500], Loss: 1.9498
Epoch [2/4], Step [6000/12500], Loss: 2.1788
Epoch [2/4], Step [8000/12500], Loss: 1.8612
Epoch [2/4], Step [10000/12500], Loss: 1.8460
Epoch [2/4], Step [12000/12500], Loss: 1.5408
Epoch [3/4], Step [2000/12500], Loss: 1.9014
Epoch [3/4], Step [4000/12500], Loss: 1.7532
Epoch [3/4], Step [6000/12500], Loss: 1.6520
Epoch [3/4], Step [8000/12500], Loss: 1.1135
Epoch [3/4], Step [10000/12500], Loss: 1.1520
Epoch [3/4], Step [12000/12500], Loss: 1.6639
Epoch [4/4], Step [2000/12500], Loss: 1.3933
Epoch [4/4], Step [4000/12500], Loss: 1.9070
Epoch [4/4], Step 